In [24]:
from __future__ import division 
from pyomo.environ import * 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
m = AbstractModel('TFG_MMP_v1.0_February_2023')

In [26]:
m.G = Set()
m.p = Set()
m.s = Set() 
m.n = Set()
m.t = Set()
m.h = Set()
m.hf = Set()
#m.aset = Set()

In [27]:
data = DataPortal()
data.load(filename= 'g.csv', set= m.G)
data.load(filename= 'p.csv', set= m.p)
data.load(filename= 's.csv', set= m.s)
data.load(filename= 'n.csv', set= m.n)
data.load(filename= 't.csv', set= m.t)
data.load(filename= 'h.csv', set= m.h)
data.load(filename= 'hf.csv', set= m.hf)
#data.load(filename= 'aset.csv', set= m.aset)

In [28]:
m.e = 480

In [29]:
#m.final = Param(m.p, m.aset)


m.CO2rate = Param(m.G, within = NonNegativeReals)
m.alfa = Param(m.G, within = NonNegativeReals)
m.beta = Param(m.G, within = NonNegativeReals)
m.gamma = Param(m.G, within = NonNegativeReals)
m.f = Param(m.G, within = NonNegativeReals)
m.o = Param(m.G, within = NonNegativeReals)
m.qmax = Param(m.G, within = NonNegativeReals)
m.qmin = Param(m.G, within = NonNegativeReals)
m.bmax = Param(m.G, within = NonNegativeReals)
m.wmax = Param(m.G)
m.w0 = Param(m.G)
m.wmin = Param(m.G)
m.k = Param(m.G)
m.rend = Param(m.G)

# Read CSV file into pandas DataFrame and convert to Pyomo parameter
param_data = pd.read_csv('Data_duration_demand1.csv', header=0, index_col=[0, 1, 2])
#model.p = Param(model.I, model.J, model.K, initialize=param_data['p'].to_dict())
m.a = Param(m.n, m.s, m.p, initialize=param_data['a'].to_dict())
m.d = Param(m.n, m.s, m.p, initialize=param_data['d'].to_dict())

param_data2 = pd.read_csv('Data_inflows1.csv', header=0, index_col=[0, 1])
m.i = Param(m.h, m.p, initialize=param_data2['i'].to_dict())


param_data3 = pd.read_csv('Data_qred1.csv', header=0, index_col=[0, 1, 2, 3])
m.qred = Param(m.t, m.s, m.n, m.p, initialize=param_data3['qred'].to_dict())
#m.p_type = Param(m.G) 

In [30]:
m.u = Var(m.t, m.s, m.p,within= Binary) #u(t,s,p) Decision to connect generator g
m.y = Var(m.t,m.s, m.p, within= Binary)#y(t,s,p) Start up decision for generator g
#m.z = Var(m.G, within= Binary)#z(p,s,g) Shut down decision for generator g #IT IS NOT USED IN THE MODEL

In [31]:
m.qt = Var(m.t, m.n, m.s, m.p) #q(t,n,s,p) Net power delivered by thermal generator t [GW]
m.qh = Var(m.h, m.n, m.s, m.p) #q(h,n,s,p) Net power delivered by hydraulic generator h [GW]
m.b = Var(m.h, m.n, m.s, m.p) #b(h,n,s,p)Gross power consumed by pumping-storage plant [GW]
m.w = Var(m.h, m.p) #w(h,p) Hydroelectric energy in the reservoir of the plant [GWh]

In [32]:
def obj_rule (m):
    return sum((m.f[t]*(m.gamma[t]*m.y[t,s,p] + sum((m.a[n, s, p] * (m.alfa[t] * m.qt[t,n,s,p] / m.k [t] + m.beta[t] * m.u[t, s, p])) for n in m.n))  + 
        sum((m.a [n, s, p]* m.o[t] * m.qt[t, n, s, p] / m.k[t]) for n in m.n )) for t in m.t for p in m.p for s in m.s)

m.obj = Objective(rule= obj_rule, sense= minimize)

In [37]:
#DECLARATION OF CONSTRAINTS (meaning of warnings????)
    #Demand balance and marginal cost
'''
    def Demand_Bal_rule(model,t,h,n,s,p):
    return sum((m.qt[t,n,s,p]) for t in m.t) + sum((m.qh[h,n,s,p] - m.b[h,n,s,p]) for h in m.h) == m.d[n,s,p]
m.Demand_Bal = Constraint(m.t, m.h, m.n, m.s, m.p, rule= Demand_Bal_rule)
'''

    #Upper thermal limit 
def Upper_Thermal_rule(model,t,n,s,p):
    return m.qt[t,n,s,p] <= m.u[t,n,s,p] * m.k[t] * m.qmax[t]

m.Upper_Thermal = Constraint(m.t, m.n, m.s, m.p, rule= Upper_Thermal_rule)
    #Upper hydro constraints
def Upper_Hydro_rule(model,h,n,s,p):
    return m.qh[h,n,s,p] <= m.k[h] * m.qmax[h]

m.Upper_Hydro = Constraint(rule= Upper_Hydro_rule)
    #Maximum Capacity limits for pumped storage
def Cap_Hydro_rule(model,h,n,s,p):
    return m.b[h,n,s,p] <= m.k[h] * m.bmax[h]

m.Cap_Hydro = Constraint(rule= Cap_Hydro_rule)
    #Lower thermal limit
def Lower_Thermal_rule(model,t,n,s,p):
    return m.qt[t,n,s,p] >= m.u[t,n,s,p] * m.k[t] * m.qmin[t]

m.Lower_Thermal = Constraint(rule= Lower_Thermal_rule)
    #Energy balance in an equivalent reservoir
def E_Bal_rule(model,h,n,s,p):
    return m.w[h,p] + sum(m.a[n,s,p] * (m.qh[h,n,s,p] - m.rend[h] * m.b[h,n,s,p]) for n in m.n for s in m.s) <= m.w[h,p - 1] + m.i[h,p]

m.E_Bal = Constraint(rule= E_Bal_rule)
    #Upper limit equivalent energy stored in reservoir
def Upper_W_rule(model,h,p):
    return m.w[h,p] <= m.wmax[h,p]

m.Upper_W = Constraint(rule = Upper_W_rule)
    #Lower limit equivalent energy stored in basin
def Lower_W_rule(model,h,p):
    return m.w[h,p] >= m.wmin[h,p]

m.Lower_W = Constraint(rule = Lower_W_rule)
    #Minimum operating hours during the year
def Min_H_rule(model,t,n,s,p):
    return sum((m.a[n,s,p] * m.qt[t,n,s,p]) for n in m.n for s in m.s for p in m.p) >= m.k[t] * m.qmax[t] * m.e

m.Min_H = Constraint( rule = Min_H_rule)
    #Minimum production to accommodate grid constraints
def Min_Grid_rule(model,t,n,s,p):
    return m.qt[t,n,s,p] >= m.qred[t,n,s,p]

m.Min_Grid = Constraint(rule = Min_Grid_rule)

    (type=<class 'pyomo.core.base.constraint.AbstractScalarConstraint'>) on
    block 'TFG_MMP_v1.0_February_2023' with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.AbstractScalarConstraint'>) on block
    'TFG_MMP_v1.0_February_2023' with a new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.AbstractScalarConstraint'>) on block
    'TFG_MMP_v1.0_February_2023' with a new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    

In [38]:
workPath = os.getcwd()

In [39]:
print(data['G'],'\n')
print(data['p'],'\n')
print(data['s'],'\n')
print(data['n'],'\n')
print(data['t'],'\n')
print(data['h'],'\n')
print(data['hf'],'\n')
#print(data['aset'],'\n')

['GEN_001', 'GEN_002', 'GEN_003', 'GEN_004', 'GEN_005', 'GEN_006', 'GEN_007', 'GEN_008', 'HYDRO_RES', 'HYDRO_ROR', 'HYDRO_PUM'] 

['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12'] 

['lab', 'fes'] 

['n1', 'n2', 'n3', 'n4', 'n5'] 

['GEN_001', 'GEN_002', 'GEN_003', 'GEN_004', 'GEN_005', 'GEN_006', 'GEN_007', 'GEN_008'] 

['HYDRO_RES', 'HYDRO_ROR', 'HYDRO_PUM'] 

['HYDRO_PUM'] 



In [40]:
data.load(filename="Data_generators1.csv", param=(m.CO2rate, m.alfa , m.beta , m.gamma , m.f , m.o , m.qmax , m.qmin , m.bmax , m.wmax , m.w0 , m.wmin , m.k , m.rend))
#why it generates problems to load more than one csv???????
#data.load(filename="Data_duration1.csv", set= (m.n, m.s, m.p))

instance = m.create_instance(data)
instance.pprint()

ERROR: Rule failed when generating expression for Constraint Upper_Thermal
    with index ('GEN_001', 'n1', 'lab', 'p1'): ValueError: Error retrieving
    component qt[GEN_001,n1,lab,p1]: The component has not been constructed.
ERROR: Constructing component 'Upper_Thermal' from data=None failed:
        ValueError: Error retrieving component qt[GEN_001,n1,lab,p1]: The
        component has not been constructed.


ValueError: Error retrieving component qt[GEN_001,n1,lab,p1]: The component has not been constructed.